In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.use("pgf")
mpl.rcParams.update({
    "pgf.texsystem": "pdflatex",
    "text.usetex": True,
    "pgf.rcfonts": False,
    "font.family": "sans-serif",
    "font.size": 9,               # testi compatti per slide 12pt
    "axes.titlesize": 10,
    "axes.labelsize": 9,
    "legend.fontsize": 8,
    "xtick.labelsize": 8,
    "ytick.labelsize": 8,
    "text.latex.preamble": r"""
\usepackage{sansmath}
\sansmath
\providecommand{\mathdefault}[1]{#1}
\providecommand{\textdefault}[1]{#1}
"""
})
import pandas as pd

In [2]:
threads = pd.read_csv("threads.csv")

threads.head()

,size_x,size_y,Ntasks,Niterations,total_time,comm_time,comp_time,slurm_job_id,slurm_job_name,partition,nodes,cpus_per_task,tasks_per_node,ntasks,omp_threads
0,16384,16384,1,500,6.414015,0.000122,6.413418,21778350,omp_scaling_112t,NaN,1,112,1,1,112
1,16384,16384,1,500,8.421359,0.000045,8.420848,21778349,omp_scaling_84t,NaN,1,84,1,1,84
2,16384,16384,1,500,12.419889,0.000094,12.419446,21778348,omp_scaling_56t,NaN,1,56,1,1,56
3,16384,16384,1,500,21.706531,0.000056,21.706215,21778347,omp_scaling_32t,NaN,1,32,1,1,32
4,16384,16384,1,500,43.212054,0.000040,43.211781,21778346,omp_scaling_16t,NaN,1,16,1,1,16


In [3]:
# Sort the dataframe by 'omp_threads' to ensure the plot is drawn correctly
threads_sorted = threads.sort_values(by='omp_threads')

# Create a plot
plt.figure(figsize=(4, 2.5))

# Plot total_time, comm_time, and comp_time as a function of omp_threads
plt.plot(threads_sorted['omp_threads'], threads_sorted['total_time'], marker='o', linestyle='-', label='Total Time')
plt.plot(threads_sorted['omp_threads'], threads_sorted['comm_time'], marker='o', linestyle='-', label='Communication Time')
plt.plot(threads_sorted['omp_threads'], threads_sorted['comp_time'], marker='o', linestyle='-', label='Computation Time')

# Add titles and labels
plt.title('Execution Time vs. Number of OMP Threads')
plt.xlabel('Number of OMP Threads')
plt.ylabel('Time (seconds)')
plt.legend()
plt.grid(True)
plt.xscale('log', base=2) # Use a log scale for the x-axis as threads are powers of 2
# plt.yscale('log') # Use a log scale for the y-axis to see all lines clearly
plt.xticks(threads_sorted['omp_threads'], threads_sorted['omp_threads']) # Ensure all thread counts are shown as ticks
plt.savefig('../slide/images/threads_sp.pgf')

In [4]:
# Calculate Speedup and Efficiency
# Serial time is the time taken with 1 OMP thread
serial_time = threads_sorted[threads_sorted['omp_threads'] == 1]['total_time'].iloc[0]

# Speedup = Serial Time / Parallel Time
threads_sorted['speedup'] = serial_time / threads_sorted['total_time']

# Efficiency = Speedup / Number of Threads
threads_sorted['efficiency'] = threads_sorted['speedup'] / threads_sorted['omp_threads']

# Display the dataframe with the new columns
threads_sorted[['omp_threads', 'total_time', 'speedup', 'efficiency']]

,omp_threads,total_time,speedup,efficiency
8,1,194.144426,1.000000,1.000000
7,2,112.609094,1.724056,0.862028
6,4,67.491750,2.876565,0.719141
5,8,53.840150,3.605941,0.450743
4,16,43.212054,4.492830,0.280802
3,32,21.706531,8.944056,0.279502
2,56,12.419889,15.631736,0.279138
1,84,8.421359,23.053812,0.274450
0,112,6.414015,30.268783,0.270257


In [5]:
# Create a plot for Speedup
plt.figure(figsize=(12, 8))
plt.plot(threads_sorted['omp_threads'], threads_sorted['speedup'], marker='o', linestyle='-', label='Measured Speedup')
# Plot ideal speedup for comparison
plt.plot(threads_sorted['omp_threads'], threads_sorted['omp_threads'], marker='x', linestyle='--', label='Ideal Speedup')
plt.title('Speedup vs. Number of OMP Threads')
plt.xlabel('Number of OMP Threads')
plt.ylabel('Speedup')
plt.xscale('log', base=2)
plt.yscale('log')
plt.xticks(threads_sorted['omp_threads'], threads_sorted['omp_threads'])
plt.legend()
plt.grid(True)
plt.show()


# Create a plot for Efficiency
plt.figure(figsize=(12, 8))
plt.plot(threads_sorted['omp_threads'], threads_sorted['efficiency'], marker='o', linestyle='-', label='Measured Efficiency')
plt.plot(threads_sorted['omp_threads'], np.ones_like(threads_sorted['omp_threads']), marker='x', linestyle='--', label='Ideal Efficiency')

plt.title('Efficiency vs. Number of OMP Threads')
plt.xlabel('Number of OMP Threads')
plt.ylabel('Efficiency')
plt.xscale('log', base=2)
plt.xticks(threads_sorted['omp_threads'], threads_sorted['omp_threads'])
plt.grid(True)
plt.legend()

plt.ylim(0, 1.1) # Efficiency is typically between 0 and 1
plt.show()

/tmp/ipykernel_17669/14765507.py:14: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()
/tmp/ipykernel_17669/14765507.py:31: UserWarning: FigureCanvasPgf is non-interactive, and thus cannot be shown
  plt.show()


In [6]:
fig, axes = plt.subplots(1, 2, figsize=(6.5, 3.2), constrained_layout=True)

# Subplot 1: Speedup
ax = axes[0]
ax.plot(threads_sorted['omp_threads'], threads_sorted['speedup'],
        marker='o', linestyle='-', label='Measured')
ax.plot(threads_sorted['omp_threads'], threads_sorted['omp_threads'],
        marker='x', linestyle='--', label='Ideal')
ax.set_title('Speedup vs. OMP Threads')
ax.set_xlabel('OMP Threads')
ax.set_ylabel('Speedup')
ax.set_xscale('log', base=2)
ax.set_yscale('log', base=2)
ax.set_yticks(threads_sorted['omp_threads'],threads_sorted['omp_threads'])
ax.set_xticks(threads_sorted['omp_threads'],threads_sorted['omp_threads'])
ax.grid(True, which='both', linestyle=':')
ax.legend()

# Subplot 2: Efficiency
ax = axes[1]
ax.plot(threads_sorted['omp_threads'], threads_sorted['efficiency'],
        marker='o', linestyle='-', label='Measured')
ax.plot(threads_sorted['omp_threads'], np.ones_like(threads_sorted['omp_threads']),
        marker='x', linestyle='--', label='Ideal')
ax.set_title('Efficiency vs. OMP Threads')
ax.set_xlabel('OMP Threads')
ax.set_ylabel('Efficiency')
ax.set_xscale('log', base=2)
ax.set_xticks(threads_sorted['omp_threads'],threads_sorted['omp_threads'])
ax.set_ylim(0, 1.1)
ax.grid(True, which='both', linestyle=':')
ax.legend()

# Salva PGF con bounding box stretto
fig.savefig("../slide/images/threads_speed_eff.pgf", bbox_inches="tight", pad_inches=0.02)
plt.close(fig)